In [2]:
# -*- coding: utf-8 -*-

# GNU nano 2.9.3                                        main.py

'''
IMPORTANT: Execute as sudo
copyright@Bene
License GPLv3
'''
import serial
import time
import cv2
import csv
import numpy as np
import time, datetime, os
import xystepper as xy
import bufcam as bufcam

# https://blog.miguelgrinberg.com/post/fun-with-the-arduino-esplora-a-digital-picture-frame

# define your positionfile here
mycsvfile = './DATA/Fermat_FOV1200um_step72um_202points.csv'
# Initialize the USB-Serial connection
try:
    serial = serial.Serial("/dev/ttyUSB0",9600)
    time.sleep(1) # connect to ARduino
except:
    print("Arduino not connected")
    

# camera paramters 
myexp_time = 500000 # 180849 #200000  # min=177293 max=2147483647 step=5434 default=18137798 value=18136663 flags=slider
myexp_time = input('Enter a exposure time (in milliseconds, default=500ms): ')
myexp_time=1000*int(myexp_time)

mycamera = '/dev/video0'
# parameters for the x/y stage 
stepsize = 17.27 # One STEPSIZE in X/Y of the cheap-stage  is 17.27 µm
myoffsetx = 10  # offset steps for the x dirrection
myoffsety = 20 # offset steps for the y dirrection
mybacklashx = 7
mybacklashy = 50 # this is required to offset the stage from the non-moving rim

# create folder for measurements
mydatafolder = './RESULTS/'
try: 
	os.mkdir(mydatafolder)	
except: 
	print('Folder already exists')
todaystr = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
myfolder = mydatafolder + 'ptycho_'+todaystr
try: 
	os.mkdir(myfolder)	
except:
    print('Error in creating the folder')


Arduino not connected
Enter a exposure time (in milliseconds, default=500ms): 12000
Folder already exists
Error in creating the folder


In [4]:
# Initilizae the Camera
print('Opening camera: '+mycamera+' with t_exp='+str(1000000/1e6)+'s')
#cap = cv2.VideoCapture('/dev/video0', cv2.CAP_V4L2)
#cap = cv2.VideoCapture('gst-launch-1.0 v4l2src device=/dev/video0 ! video/x-raw, format=BGRx ! ximagesink')#
mycam_cmd = 'v4l2src device='+ mycamera+ ' extra-controls='+'"'+'c,exposure='+str(myexp_time)+',exposure_auto=1'+'"' +' ! video/x-raw, format=BGRx ! videoconvert ! appsink'
#v4l2src device=/dev/video0 extra-controls="c,exposure=200000,exposure_auto=1" ! video/x-raw, format=BGRx ! ximagesink

print(mycam_cmd)
# open camera object and display object
#cap = cv2.VideoCapture(mycam_cmd, cv2.CAP_GSTREAMER)
cap = bufcam.VideoCapture(mycam_cmd) 
#cap.set(cv2.CAP_PROP_CONVERT_RGB, False);	# Request raw camera data
#print("Buffsize:" + cap.set(cv2.CAP_PROP_BUFFERSIZE))		
#cap.set(cv2.CAP_PROP_BUFFERSIZE, 3)		# Limit buffer size to get the exact frame

window_handle = cv2.namedWindow("CSI Camera", cv2.WINDOW_AUTOSIZE)

#ake a background image first! 
print("Blank the beam for five seconds") 
time.sleep(5)
# grab a frame and wait until the camera settles
print('Darkframe was acquired frame')


Opening camera: /dev/video0 with t_exp=1.0s
v4l2src device=/dev/video0 extra-controls="c,exposure=12000000,exposure_auto=1" ! video/x-raw, format=BGRx ! videoconvert ! appsink
Blank the beam for five seconds
Darkframe was acquired frame


In [9]:
print('Darkframe was acquired frame')
img = cap.read()
print('Remove the Beamblocker')
# write out the image information
cap.cap.set(cv2.CAP_PROP_CONVERT_RGB, False);          # Request raw camera data

if(True):
	#print(ret_val)
	frame_16 = img.view(dtype=np.int16)       # reinterpret data as 16-bit pixels
	frame_sh = np.right_shift(frame_16, 2)      # Shift away the bottom 2 bits
	frame_8  = frame_sh.astype(np.uint8)        # Keep the top 8 bits       
	img      = frame_8.reshape(img.shape[0], img.shape[1])        # Arrange them into a rectangle

cv2.imwrite(myfolder+'/background.tif', img)
try:
    cv2.imshow("CSI Camera", img)
except:
    print('Could not display frame?!')



Darkframe was acquired frame
Remove the Beamblocker


ValueError: When changing to a larger dtype, its size must be a divisor of the total size in bytes of the last axis of the array.